# 실습 10: Cursor AI를 활용한 Document AI 토이 프로젝트

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab10_토이프로젝트.ipynb)

## 🎯 학습 목표
- Cursor AI 활용법 이해
- 실무 프로젝트 구조 설계
- 엔드투엔드 Document AI 시스템 구축

## ⏱️ 소요 시간: 60분
## 📊 난이도: ⭐⭐⭐⭐⭐

---

> ⚠️ **참고**: 이 실습은 로컬 Cursor AI 환경에서 진행합니다.
> Colab에서는 Cursor 없이 진행 가능한 예제를 제공합니다.


## 📋 프로젝트 구조

```
document_processor/
├── app.py                 # Streamlit 앱
├── processor/
│   ├── __init__.py
│   ├── ocr_engine.py      # OCR 처리
│   ├── preprocessor.py    # 전처리
│   └── ner_extractor.py   # 정보 추출
├── data/
│   ├── input/            # 입력 이미지
│   └── output/           # 결과 저장
└── requirements.txt
```


In [ ]:
# Colab에서 간단한 Document Processor 구현
!pip install -q streamlit paddleocr

import streamlit as st
from paddleocr import PaddleOCR
import cv2
import pandas as pd
import re

print("✅ 환경 설정 완료!")


In [ ]:
# Document Processor 클래스 (통합)
class DocumentProcessor:
    def __init__(self):
        self.ocr = PaddleOCR(use_textline_orientation=True, lang='korean')
        
    def process(self, image_path):
        """이미지 처리 파이프라인"""
        results = {}
        
        # 1. OCR 실행
        ocr_result = self.ocr.predict(image_path)
        full_text = ' '.join([line[1][0] for line in ocr_result[0]])
        results['full_text'] = full_text
        
        # 2. 정보 추출 (NER)
        entities = self._extract_entities(full_text)
        results['entities'] = entities
        
        # 3. 신뢰도 분석
        confidences = [line[1][1] for line in ocr_result[0]]
        results['avg_confidence'] = sum(confidences) / len(confidences) if confidences else 0
        
        return results
    
    def _extract_entities(self, text):
        """규칙 기반 정보 추출"""
        entities = {}
        
        # 날짜
        date_pattern = r'(\\d{4})년\\s*(\\d{1,2})월\\s*(\\d{1,2})일'
        date_match = re.search(date_pattern, text)
        if date_match:
            entities['날짜'] = date_match.group(0)
        
        # 금액
        amount_pattern = r'([\\d,]+)원'
        amount_match = re.search(amount_pattern, text)
        if amount_match:
            entities['금액'] = amount_match.group(0)
        
        # 전화번호
        phone_pattern = r'(\\d{2,3}-\\d{3,4}-\\d{4})'
        phone_match = re.search(phone_pattern, text)
        if phone_match:
            entities['전화번호'] = phone_match.group(0)
        
        # 이메일
        email_pattern = r'([\\w\\.-]+@[\\w\\.-]+\\.[\\w]+)'
        email_match = re.search(email_pattern, text)
        if email_match:
            entities['이메일'] = email_match.group(0)
        
        return entities

processor = DocumentProcessor()
print("✅ Document Processor 준비 완료!")


In [ ]:
# 샘플 계약서 이미지 생성 및 테스트
from PIL import Image, ImageDraw

img = Image.new('RGB', (700, 400), color='white')
draw = ImageDraw.Draw(img)

contract_text = """
용역 계약서

계약일자: 2025년 3월 15일
계약금액: 25,000,000원

갑: 삼성전자 주식회사
  담당자: 김철수 부장
  연락처: 02-1234-5678
  이메일: chulsoo@samsung.com

을: (주)테크솔루션
  담당자: 이영희 대리
  연락처: 031-9876-5432
  이메일: younghee@techsolution.com
"""

draw.text((30, 30), contract_text, fill='black')
img.save('contract_sample.jpg')

# 처리 실행
print("📄 계약서 처리 중...\\n")
results = processor.process('contract_sample.jpg')

print("="*60)
print("📊 처리 결과")
print("="*60)
print(f"\\n평균 신뢰도: {results['avg_confidence']:.2%}\\n")
print("추출된 정보:")
for key, value in results['entities'].items():
    print(f"  • {key}: {value}")
    
print(f"\\n전체 텍스트:\\n{results['full_text']}")
print("="*60)
print("\\n✅ 문서 처리 완료!")
